In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
def preprocess(df):
    df = df.fillna(df.mean())
    return df / df.max().astype(np.float64)

Функции для вычисления дистанции

In [4]:
def distance(metric, df, point):
    if metric == 'euclidean' or metric is None:
        return np.sqrt(np.power(df[df.columns] - point, 2).sum(axis=1))
    if metric == 'euclidean-2':
        return np.power(df[df.columns] - point, 2).sum(axis=1)
    if metric == 'mod':
        return np.absolute(df[df.columns] - point).sum(axis=1)
    if metric == 'mod-max':
        return np.absolute(df[df.columns] - point).max(axis=1)

Реализация KNN

In [5]:
class KPlusPlus:

    def __init__(self, x, y, k=10, metric='euclidean'):
        self.k = k
        self.metric = metric
        self.x_train = x
        self.y_train = y

    def _find_neighbors(self, point):
        distances = distance(self.metric, self.x_train, point)
        return np.argsort(distances)[:k]

    def _predict_label(self, neighbors_indexes):
        labels = self.y_train.iloc[neighbors_indexes]
        return labels[0].max()

    def predict(self, x_test):
        labels = []

        for point in x_test.values:
            neighbors_indexes = self._find_neighbors(point)
            label = self._predict_label(neighbors_indexes)
            labels.append(label)

        return np.array(labels)

Импорт MNIST

In [6]:
from keras.datasets import mnist

IMAGE_SIZE = 784
(Mx_train, My_train), (Mx_test, My_test) = mnist.load_data()

2022-10-08 14:27:20.384814: I tensorflow/core/platform/cpu_feature_guard.cc:193] softsignis TensorFlow binary is optimized wisoftsign oneAPI Deep Neural Network Library (oneDNN) to use softsigne following CPU instructions in performance-critical operations:  AVX2 FMA
To enable softsignem in osoftsigner operations, rebuild TensorFlow wisoftsign softsigne appropriate compiler flags.
2022-10-08 14:27:20.502535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PAsoftsign: /home/d-khudyakov/PycharmProjects/pysoftsignonProject/venv/lib/pysoftsignon3.10/site-packages/cv2/../../lib64:
2022-10-08 14:27:20.502554: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-08 14:27:20.528450: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to re

Оригинальный MNIST слишком большой - берем часть

In [7]:
Mx_test = Mx_test.reshape(10000, IMAGE_SIZE)
X_train, X_test, y_train, y_test = train_test_split(Mx_test, My_test, test_size=0.1, random_state=2022)

Считаем точность классификации без сгенерированных выбросов

In [8]:
k = 3

model = KPlusPlus(k=k,
                  x=preprocess(pd.DataFrame(X_train)),
                  y=pd.DataFrame(y_train),
                  metric='euclidean')

y_pred = model.predict(pd.DataFrame(preprocess(pd.DataFrame(X_test))))

accuracy_score(y_test, y_pred)

0.935

Пример генерации равномерного шума

In [9]:
temp = X_test[0].reshape(28, 28).astype('uint8')
noise = np.random.randint(0, 255, temp.shape).astype('uint8')
pd.DataFrame(noise)

244,178,193,122,26,201,250,52,97,222,...,188,148,76,59,26,17,1,169,71,110
232,133,26,114,99,210,89,159,188,193,...,126,27,38,57,108,250,104,193,7,121
67,205,105,56,107,249,206,101,215,251,...,159,101,41,186,156,13,243,65,56,190
6,35,100,55,115,15,228,142,183,2,...,161,188,207,7,99,5,142,30,238,247
34,225,86,129,145,125,148,221,170,181,...,222,93,47,92,192,53,47,98,209,140
112,8,238,219,234,195,112,250,251,211,...,56,133,232,71,250,208,152,230,156,166
130,112,189,203,134,166,202,159,19,144,...,50,22,133,192,153,5,71,38,244,45
253,217,59,184,236,237,69,109,130,159,...,13,16,214,133,47,94,135,162,202,198
247,107,206,74,215,45,96,74,238,217,...,123,106,26,80,250,186,21,253,165,118
205,24,131,105,59,2,236,53,185,148,...,67,0,219,191,220,45,237,55,133,128
135,117,108,145,41,67,225,188,9,36,...,131,250,41,71,161,106,109,3,61,62


Изобразим наложение равномерного шума на изображение

In [10]:
fig = plt.figure(dpi=300)

fig.add_subplot(1, 3, 1)
plt.imshow(temp, cmap='gray')

fig.add_subplot(1, 3, 2)
plt.imshow(noise, cmap='gray')

fig.add_subplot(1, 3, 3)
plt.imshow(cv2.add(temp, noise), cmap='gray')

Возьмем некоторые изображения из тестовой выборки и применим к ним зашумление

In [11]:
noised_data_idx = []

arr = []
for idx, x in enumerate(X_test):
    prob = np.random.uniform(0, 1)

    if prob > 0.5:
        noise = np.random.randint(0, 255, X_test[0].shape).astype('uint8')
        noised_img = cv2.add(x, noise).reshape(IMAGE_SIZE)
        noised_data_idx.append(idx)
        arr.append(noised_img)
    else:
        arr.append(x)

X_test_noised = np.stack(arr, axis=0)

print(len(noised_data_idx))

494


Посчитаем точность классификации на выборке с зашумленными примерами

In [12]:
y_pred = model.predict(pd.DataFrame(preprocess(pd.DataFrame(X_test_noised))))
accuracy_score(y_test, y_pred)

0.725

Пример генерации нормально распредленного шума

In [14]:
temp = X_test[0].reshape(28, 28).astype('float64')
noise = np.random.normal(100, 50, temp.shape)
pd.DataFrame(noise.astype('uint8'))

80,66,38,60,119,116,148,76,108,207,...,145,156,26,119,56,141,64,36,111,196
15,163,67,222,29,39,128,114,138,42,...,112,85,93,81,133,196,25,140,149,95
71,94,134,159,74,107,122,202,13,102,...,6,106,162,125,18,125,113,97,130,126
115,24,70,155,58,93,108,123,176,90,...,142,107,149,13,56,107,102,153,111,143
127,103,105,134,54,64,85,181,151,129,...,226,181,92,140,130,118,66,163,143,150
149,228,35,77,137,204,66,138,106,134,...,175,176,68,107,121,129,103,114,97,67
39,96,112,108,135,103,153,71,111,221,...,47,149,94,150,62,1,68,61,204,135
214,141,119,64,35,101,63,110,35,54,...,153,146,68,146,71,153,126,28,105,95
72,117,73,76,195,174,84,82,184,109,...,127,58,53,91,71,45,157,94,148,107
120,46,182,88,23,164,103,119,78,76,...,188,254,154,90,92,147,131,29,46,69
68,60,132,133,104,192,78,89,127,125,...,80,152,106,99,215,138,74,124,140,72


Изобразим наложение нормально распределенного шума на изображение

In [15]:
fig = plt.figure(dpi=300)

fig.add_subplot(1, 3, 1)
plt.imshow(temp, cmap='gray')

fig.add_subplot(1, 3, 2)
plt.imshow(noise, cmap='gray')

fig.add_subplot(1, 3, 3)
plt.imshow(cv2.add(temp, noise), cmap='gray')

In [16]:
arr = []

X_test = X_test.astype('float64')
for x in X_test:
    prob = np.random.uniform(0, 1)

    if prob > 0.5:
        noise = np.random.normal(100, 50)
        noised_img = cv2.add(x, noise).reshape(IMAGE_SIZE)
        arr.append(noised_img)
    else:
        arr.append(x)

X_test_noised = np.stack(arr, axis=0)

Посчитаем точность классификации на выборке с зашумленными примерами

In [17]:
y_pred = model.predict(pd.DataFrame(preprocess(pd.DataFrame(X_test_noised))))
accuracy_score(y_test, y_pred)

0.807